# Tutorial: "hello world!" (Part 1 of 3)

---
## Introduction
In **part 1 of this get started series**, you will submit a trivial "hello world" python script to the cloud by:

- Running Python code in the cloud with Azure Machine Learning SDK
- Switching between debugging locally on a compute instance
- Submitting remote runs in the cloud
- Monitoring and recording runs in the Azure Machine Learning studio


### Test in your development environment

You can test your code works on a compute instance or locally (for example, a laptop), which has the benefit of interactive debugging of code:

In [ ]:
!python src/hello.py

## Submit your code to Azure Machine Learning

Below you create a __*control script*__ this is where you specify _how_ your code is submitted to Azure Machine Learning. The code you submit to Azure Machine Learning (in this case `hello.py`) does not need anything specific to Azure Machine Learning - it can be any valid Python code. It is only the control script that is Azure Machine Learning specific.

The code below will show a Jupyter widget that tracks the progress of your run, and displays logs.

> <span style="color:purple; font-weight:bold">! NOTE <br>
> The very first run will take 5-10 minutes to complete. This is because in the background a docker image is built in the cloud, the compute cluster is resized from 0 to 1 node, and the docker image is downloaded to the compute. Subsequent runs are much quicker (~15 seconds) as the docker image is cached on the compute - you can test this by resubmitting the code below after the first run has completed.</span>

In [4]:
#from azureml.core import Workspace, Experiment, ScriptRunConfig
#from azureml.widgets import RunDetails

#ws = Workspace.from_config()
#exp = Experiment(workspace=ws, name="an-introduction-hello-world-tutorial")
#src = ScriptRunConfig(
#    source_directory="src", script="hello.py", compute_target="cpu-cluster"
#)

#run = exp.submit(src)
#RunDetails(run).show()

subscription_id = '5f08d643-1910-4a38-a7c7-84a39d4f42e0'
resource_group = 'hanchi-test'
workspace = 'hanchi-test'

from azure.ml import MLClient
from azure.ml.entities import CommandJob
from azure.ml._schema.code_asset import InternalCodeAsset
from azure.ml._schema.compute_binding import InternalComputeBinding

ml_client = MLClient(subscription_id, resource_group, workspace)

job = CommandJob(code=InternalCodeAsset(base_path=".", directory="src"),
                 command="python hello.py",
                 compute=InternalComputeBinding(target="/subscriptions/5f08d643-1910-4a38-a7c7-84a39d4f42e0/resourceGroups/hanchi-test/providers/Microsoft.MachineLearningServices/workspaces/hanchi-test/computes/cpu-cluster"),
                 environment="/subscriptions/5f08d643-1910-4a38-a7c7-84a39d4f42e0/resourceGroups/hanchi-test/providers/Microsoft.MachineLearningServices/workspaces/hanchi-test/environments/AzureML-Minimal/versions/1")

returned_job = ml_client.jobs.create_or_update(job)
returned_job.__dict__

{'_name': '6ac5322c-c0c5-4308-af6d-68c2fb1885f5',
 '_id': '/subscriptions/5f08d643-1910-4a38-a7c7-84a39d4f42e0/resourceGroups/hanchi-test/providers/Microsoft.MachineLearningServices/workspaces/hanchi-test/jobs/6ac5322c-c0c5-4308-af6d-68c2fb1885f5',
 '_description': None,
 '_tags': {},
 '_properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '5be5a4c3-bec3-4d9f-852b-0208cfb439d4',
  'azureml.datastoreId': '/subscriptions/5f08d643-1910-4a38-a7c7-84a39d4f42e0/resourceGroups/hanchi-test/providers/Microsoft.MachineLearningServices/workspaces/hanchi-test/datastores/hanchitest5194569865_azureml',
  'mlflow.source.git.repoURL': 'https://github.com/Azure/azureml-examples.git',
  'mlflow.source.git.branch': 'v2-draft',
  'mlflow.source.git.commit': 'dc7d1f661fa2721a9eb265d10cd5bb46e64e6fb5',
  'azureml.git.dirty': 'True'},
 'input_values': {},
 '_command': 'python hello.py',
 '_input_ports': {},
 '_data_bindings': {},
 '_base_path': None,
 'compute': <azure.ml._schema.

### Understanding the control code

| Code |Description  | 
|---|---|
| `ws = Workspace.from_config()` |   [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py&preserve-view=true) connects to your Azure Machine Learning workspace, so that you can communicate with your Azure Machine Learning resources. |
| `exp =  Experiment( ... )`   | [Experiment](https://docs.microsoft.com/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py&preserve-view=true) provides a simple way to organize multiple runs under a single name. <br>Later you can see how experiments make it easy to compare metrics between dozens of runs.  |
| `src = ScriptRunConfig( ... )` |  [ScriptRunConfig](https://docs.microsoft.com/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py&preserve-view=true) wraps your `hello.py` code and passes it to your workspace.<br> As the name suggests, you can use this class to _configure_ how you want your _script_ to _run_ in Azure Machine Learning. <br>Also specifies what compute target the script will run on.  <br>In this code, the target is the compute cluster you created in the [setup tutorial](tutorial-1st-experiment-sdk-setup-local.md). |
| `run = exp.submit(config)` |  Submits your script. This submission is called a [Run](https://docs.microsoft.com/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py&preserve-view=true).  <br>A run encapsulates a single execution of your code. Use a run to monitor the script progress, capture the output,<br> analyze the results, visualize metrics and more. |
|`RunDetails(run).show()` | There is an Azure Machine Learning widget that shows the progress of your job along with streaming the log files.

## View the logs

The widget has a dropdown box titled **Output logs** select `70_driver_log.txt`, which shows the following standard output: 

```
 1: [2020-08-04T22:15:44.407305] Entering context manager injector.
 2: [context_manager_injector.py] Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError', 'UserExceptions:context_managers.UserExceptions'], invocation=['hello.py'])
 3: Starting the daemon thread to refresh tokens in background for process with pid = 31263
 4: Entering Run History Context Manager.
 5: Preparing to call script [ hello.py ] with arguments: []
 6: After variable expansion, calling script [ hello.py ] with arguments: []
 7:
 8: hello world!
 9: Starting the daemon thread to refresh tokens in background for process with pid = 31263
10:
11:
12: The experiment completed successfully. Finalizing run...
13: Logging experiment finalizing status in history service.
14: [2020-08-04T22:15:46.541334] TimeoutHandler __init__
15: [2020-08-04T22:15:46.541396] TimeoutHandler __enter__
16: Cleaning up all outstanding Run operations, waiting 300.0 seconds
17: 1 items cleaning up...
18: Cleanup took 0.1812913417816162 seconds
19: [2020-08-04T22:15:47.040203] TimeoutHandler __exit__
```

On line 8 above, you see the "Hello world!" output. The 70_driver_log.txt file contains the standard output from run and can be useful when debugging remote runs in the cloud. You can also view the run by clicking on the **Click here to see the run in Azure Machine Learning studio** link in the widget.

## Next steps

In this tutorial, you took a simple "hello world" script and ran it on Azure. You saw how to connect to your Azure Machine Learning workspace, create an Experiment, and submit your `hello.py` code to the cloud.

In the [next tutorial](2.pytorch-model.ipynb), you build on these learnings by running something more interesting than `print("hello world!")`.

In [5]:
ml_client.jobs.stream_logs(returned_job.name) # equivalent to wait_for_completion(show_output=True)

RunId: 6ac5322c-c0c5-4308-af6d-68c2fb1885f5
Web View: https://ml.azure.com/experiments/Default/runs/6ac5322c-c0c5-4308-af6d-68c2fb1885f5?wsid=/subscriptions/5f08d643-1910-4a38-a7c7-84a39d4f42e0/resourcegroups/hanchi-test/workspaces/hanchi-test

Execution Summary
RunId: 6ac5322c-c0c5-4308-af6d-68c2fb1885f5
Web View: https://ml.azure.com/experiments/Default/runs/6ac5322c-c0c5-4308-af6d-68c2fb1885f5?wsid=/subscriptions/5f08d643-1910-4a38-a7c7-84a39d4f42e0/resourcegroups/hanchi-test/workspaces/hanchi-test


Exception: Exception : 
 {
    "error": {
        "code": "ServiceError",
        "message": "AzureMLCompute job failed.\nMountError: Unable to mount Azure File or blob file system\n\tInfo: Error calling XDS API for GenerateDataStoreCredentials: InternalError - :{\n}",
        "details": []
    },
    "correlation": {
        "operation": null,
        "request": "782408ece880f4a1"
    },
    "environment": "master",
    "location": "westus2",
    "time": "2021-02-19T23:59:28.186686Z"
} 

In [ ]:
run.wait_for_completion(show_output=True)